In [1]:
import xcf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras_nlp
from keras.utils.vis_utils import plot_model

2023-05-05 13:38:17.062375: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-05 13:38:17.786276: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
AA_VOCAB = ['<PAD>','<START>','<END>','<MASK>','A','B','C','D','E','F','G','H','I','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
MAX_SEQUENCE_LENGTH = 600 #Not counting <START> and <END> tokens

BASELINE_MODEL_DIM = 256
BASELINE_BATCH_SIZE = 32
BASELINE_EPOCHS = 10

MASK_RATE = 0.25
PREDICTIONS_PER_SEQ = 32

In [3]:
data = xcf.load_data_from_db('data_pHpred_from_data_raw_2023-04-07', mode = 'clean')
data['seq_as_words'] = data['sequence'].apply(lambda s: ' '.join([char for char in s]))
data = data[['seq_as_words', 'pH']]
data.head()

,seq_as_words,pH
rcsb_id,,
4BOB,G A M G D E Q S S G E I N H T L Y D E Q S N G ...,7.5
4BOD,G A M G D G Q S N G E A K V K K I E F S E F T ...,7.5
4BOF,M T A Q T P I H V Y S E I G K L K K V L L H R ...,5.5
4BPD,G H H H H H H E L A N N T T G F T R I I K A A ...,5.6
4BPM,M H H H H H H S P A L P A F L L C S T L L V I ...,6.7


In [4]:
data_train, data_val, data_test = xcf.split_df(data)

tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=AA_VOCAB,
    sequence_length=MAX_SEQUENCE_LENGTH,
    oov_token='X'
)
data_train_X = data_train['seq_as_words'].to_numpy(dtype='str')
data_val_X = data_val['seq_as_words'].to_numpy(dtype='str')
data_test_X = data_test['seq_as_words'].to_numpy(dtype='str')
data_train_X = tokenizer(data_train_X)
data_val_X = tokenizer(data_val_X)
data_test_X = tokenizer(data_test_X)
data_train_Y = data_train['pH'].to_numpy(dtype='float')
data_val_Y = data_val['pH'].to_numpy(dtype='float')
data_test_Y = data_test['pH'].to_numpy(dtype='float')

2023-05-05 13:38:20.750486: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-05 13:38:20.750903: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [5]:
baseline_model = xcf.models.compile_phpred_baseline(tokenizer, MAX_SEQUENCE_LENGTH, BASELINE_MODEL_DIM)

In [7]:
history = baseline_model.fit(x=data_train_X, y=data_train_Y, batch_size=BASELINE_BATCH_SIZE, epochs=BASELINE_EPOCHS)

Epoch 1/10
  85/2180 [>.............................] - ETA: 1:56 - loss: 9.7365 - mean_squared_error: 9.7365

KeyboardInterrupt: 

In [11]:
results = baseline_model.evaluate(x=data_val_X, y=data_val_Y, batch_size=BASELINE_BATCH_SIZE)

4/4 [==============================] - 0s 22ms/step - loss: 1.8378
